In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from apyori import apriori

파일 불러오기

In [15]:
df = pd.read_csv('son1.csv',encoding='utf-8')
print(len(df))
df.head(3)

1000


,Unnamed: 0,created,tweet_text
0,0,Sat Jun 12 07:28:52 +0000 2021,가슴 경주 대구 마사 마산 부산 소통 야동 인천 일탈 최신영화 손흥민 만남앱 소개팅...
1,1,Sat Jun 12 07:17:25 +0000 2021,코인세탁소 산업혁명셀프 사이트 인공지능 빨래방 손흥민 그래핀 지도 강남구 광고 세탁...
2,2,Sat Jun 12 07:12:41 +0000 2021,RT @sonny_record: 210611 팬문선답 타래 #손흥민 https://...


데이터 정규화

In [16]:
import re

# 정규표현식 - 한글만 포함
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힝]+') # 한글만 포함시킴
    result = hangul.sub('',text)
    return result

In [17]:
# 한글만 포함해서 컬럼 저장
df['ko_text'] =  df['tweet_text'].apply(lambda x: text_cleaning(x))
df.head(2)

,Unnamed: 0,created,tweet_text,ko_text
0,0,Sat Jun 12 07:28:52 +0000 2021,가슴 경주 대구 마사 마산 부산 소통 야동 인천 일탈 최신영화 손흥민 만남앱 소개팅...,가슴 경주 대구 마사 마산 부산 소통 야동 인천 일탈 최신영화 손흥민 만남앱 소개팅...
1,1,Sat Jun 12 07:17:25 +0000 2021,코인세탁소 산업혁명셀프 사이트 인공지능 빨래방 손흥민 그래핀 지도 강남구 광고 세탁...,코인세탁소 산업혁명셀프 사이트 인공지능 빨래방 손흥민 그래핀 지도 강남구 광고 세탁...


Konlpy를 이용한 키워드 분할

In [18]:
from konlpy.tag import Okt
# 글자가 몇번 나왔는지 확인, 카운트 함수
from collections import Counter

#  불용어사전을 이용해서 필요없는 단어 제거
with open('korean_stopwords.txt',encoding='utf-8') as f:
    stopwords = f.readlines()
# list타입
stopwords = [x.strip() for x in stopwords]    

In [19]:
stopwords[:3]

['가', '가까스로', '가령']

In [24]:
# 형태소 분석, 한글자 삭제, 불용어 제거
def get_nouns(x):
    # 키워드연관분석을 하려면 명사만 필요
    # list타입
    nouns_tagger = Okt()
    nouns = nouns_tagger.nouns(x)
    # 한글자 키워드는 모두제거
    nouns = [noun for noun in nouns if len(noun)>1 ]
    # 불용어 제거
    nouns = [noun for noun in nouns if noun not in stopwords ]
    return nouns

In [25]:
df['nouns'] = df['ko_text'].apply(lambda x : get_nouns(x))
df.head(3)

,Unnamed: 0,created,tweet_text,ko_text,nouns
0,0,Sat Jun 12 07:28:52 +0000 2021,가슴 경주 대구 마사 마산 부산 소통 야동 인천 일탈 최신영화 손흥민 만남앱 소개팅...,가슴 경주 대구 마사 마산 부산 소통 야동 인천 일탈 최신영화 손흥민 만남앱 소개팅...,"[가슴, 경주, 대구, 마사, 마산, 부산, 소통, 야동, 인천, 최신영, 손흥민,..."
1,1,Sat Jun 12 07:17:25 +0000 2021,코인세탁소 산업혁명셀프 사이트 인공지능 빨래방 손흥민 그래핀 지도 강남구 광고 세탁...,코인세탁소 산업혁명셀프 사이트 인공지능 빨래방 손흥민 그래핀 지도 강남구 광고 세탁...,"[코인, 세탁소, 산업혁명, 셀프, 사이트, 인공, 지능, 빨래, 손흥민, 그래핀,..."
2,2,Sat Jun 12 07:12:41 +0000 2021,RT @sonny_record: 210611 팬문선답 타래 #손흥민 https://...,팬문선답 타래 손흥민,"[문선, 타래, 손흥민]"


## 키워드 연관검색

샘플을 가지고 실습

In [26]:
# 샘플
transactions = [
    ['손흥민','시소코'],
    ['손흥민','케인'],
    ['손흥민','케인','콘테'],
]

# 연관분석 모델 적용
results = list(apriori(transactions))
for result in results:
    print(result)

RelationRecord(items=frozenset({'손흥민'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥민'}), confidence=1.0, lift=1.0)])
RelationRecord(items=frozenset({'시소코'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'시소코'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'케인'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'케인'}), confidence=0.6666666666666666, lift=1.0)])
RelationRecord(items=frozenset({'콘테'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'콘테'}), confidence=0.3333333333333333, lift=1.0)])
RelationRecord(items=frozenset({'손흥민', '시소코'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'손흥민', '시소코'}), confidence=0.3333333333333333, lift=1.0), Ordere